In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import os
import pickle
import random
data_path = '../results/cluster_files/test_cluster2.pkl'

In [2]:
def get_data(data_file):
    data = pickle.load(open(data_file,'rb'))
    train_data = data['train']
    test_data = data['test']
    return (train_data, test_data)


In [3]:
def stack_data(in_dict):
    class_ids = sorted(in_dict.keys())
    class_idx = {x:ctr for ctr,x in enumerate(class_ids)}
    num_classes = len(class_ids) 

    data = [in_dict[x][0] for x in class_ids]
    labels = []
    for x in class_ids:
        num_samps, num_classes_b = in_dict[x][1].shape
        assert num_classes == num_classes_b
        label_array = np.zeros((num_samps, num_classes_b))
        curr_hot = class_idx[x]
        label_array[:,curr_hot] = 1
        labels.append(label_array)

    data = np.vstack(data)
    labels = np.vstack(labels)

    return data, labels

In [4]:
def get_dataset(data_dict, curr_epoch):
    data, labels = stack_data(data_dict [curr_epoch])

    dataset = tf.data.Dataset.from_tensor_slices((data, labels))
    dataset = dataset.shuffle(buffer_size=1024).batch(64)
    
    return dataset
    
def get_model():
    reg = tf.keras.regularizers.l2(l=0.01)

    inputs = keras.Input(shape=(640,), name="LastHidden")
    outputs = tf.keras.layers.Dense(65, activation='softmax',
                                    kernel_regularizer=reg, 
                                    name="Output")(inputs)
    model = keras.Model(inputs=inputs, outputs=outputs)
    #model.summary()

    loss_fnc = 'categorical_crossentropy'
    opt = tf.keras.optimizers.SGD(learning_rate=0.0005, momentum=0.9, nesterov=True)

    model.compile(
        optimizer=opt,
        loss=loss_fnc,
        metrics=[tf.keras.metrics.CategoricalAccuracy(name="accuracy", dtype=None)]
    )
    
    return model

In [5]:
def get_acc_vs_src_epochs(file_path):
    
    # Initialize output dict
    result = {}
    
    # Set hyper-parameters for net
    loss_fnc = 'categorical_crossentropy'
    opt = tf.keras.optimizers.SGD(learning_rate=0.0005, momentum=0.9, nesterov=True)
    metric = tf.keras.metrics.CategoricalAccuracy(name="accuracy", dtype=None)

    train_data, test_data =  get_data(file_path)
    use_data = test_data

    epochs = sorted([x for x in use_data.keys()])
    for curr_epoch in epochs:
        data, labels = stack_data(use_data[curr_epoch])
        dataset = tf.data.Dataset.from_tensor_slices((data, labels))
        dataset = dataset.shuffle(buffer_size=1024).batch(64)

        model = get_model()
        model.compile(
            optimizer=opt,
            loss=loss_fnc,
            metrics=[metric]
        )

        print("\nCurrent Epoch:",curr_epoch)
        history = model.fit(dataset, epochs=50)
        result[curr_epoch] = [history.history['accuracy'][-1], 
                              max(history.history['accuracy'])]
        
    return result

In [6]:
result = get_acc_vs_src_epochs(data_path)


Current Epoch: 0
Epoch 1/50
102/102 [==============================] - 0s 3ms/step - loss: 5.3403 - accuracy: 0.0118
Epoch 2/50
102/102 [==============================] - 0s 3ms/step - loss: 5.3169 - accuracy: 0.0102
Epoch 3/50
102/102 [==============================] - 0s 3ms/step - loss: 5.2938 - accuracy: 0.0111
Epoch 4/50
102/102 [==============================] - 0s 3ms/step - loss: 5.2712 - accuracy: 0.0151
Epoch 5/50
102/102 [==============================] - 0s 3ms/step - loss: 5.2490 - accuracy: 0.0123
Epoch 6/50
102/102 [==============================] - 0s 3ms/step - loss: 5.2273 - accuracy: 0.0114
Epoch 7/50
102/102 [==============================] - 0s 3ms/step - loss: 5.2061 - accuracy: 0.0114
Epoch 8/50
102/102 [==============================] - 0s 3ms/step - loss: 5.1852 - accuracy: 0.0128
Epoch 9/50
102/102 [==============================] - 0s 3ms/step - loss: 5.1648 - accuracy: 0.0117
Epoch 10/50
102/102 [==============================] - 0s 3ms/step - loss: 5.1448 

102/102 [==============================] - 0s 3ms/step - loss: 3.1357 - accuracy: 0.4878
Epoch 32/50
102/102 [==============================] - 0s 3ms/step - loss: 3.1123 - accuracy: 0.4894
Epoch 33/50
102/102 [==============================] - 0s 3ms/step - loss: 3.0903 - accuracy: 0.4918
Epoch 34/50
102/102 [==============================] - 0s 3ms/step - loss: 3.0682 - accuracy: 0.4952
Epoch 35/50
102/102 [==============================] - 0s 3ms/step - loss: 3.0473 - accuracy: 0.4949
Epoch 36/50
102/102 [==============================] - 0s 3ms/step - loss: 3.0274 - accuracy: 0.4982
Epoch 37/50
102/102 [==============================] - 0s 3ms/step - loss: 3.0089 - accuracy: 0.5023
Epoch 38/50
102/102 [==============================] - 0s 3ms/step - loss: 2.9906 - accuracy: 0.5032
Epoch 39/50
102/102 [==============================] - 0s 3ms/step - loss: 2.9723 - accuracy: 0.5055
Epoch 40/50
102/102 [==============================] - 0s 3ms/step - loss: 2.9553 - accuracy: 0.5092
Ep

102/102 [==============================] - 0s 3ms/step - loss: 3.4803 - accuracy: 0.4780
Epoch 13/50
102/102 [==============================] - 0s 3ms/step - loss: 3.4020 - accuracy: 0.4937
Epoch 14/50
102/102 [==============================] - 0s 3ms/step - loss: 3.3319 - accuracy: 0.5112
Epoch 15/50
102/102 [==============================] - 0s 3ms/step - loss: 3.2673 - accuracy: 0.5188
Epoch 16/50
102/102 [==============================] - 0s 3ms/step - loss: 3.2098 - accuracy: 0.5246
Epoch 17/50
102/102 [==============================] - 0s 3ms/step - loss: 3.1549 - accuracy: 0.5368
Epoch 18/50
102/102 [==============================] - 0s 3ms/step - loss: 3.1058 - accuracy: 0.5420
Epoch 19/50
102/102 [==============================] - 0s 3ms/step - loss: 3.0601 - accuracy: 0.5498
Epoch 20/50
102/102 [==============================] - 0s 3ms/step - loss: 3.0177 - accuracy: 0.5511
Epoch 21/50
102/102 [==============================] - 0s 3ms/step - loss: 2.9781 - accuracy: 0.5585
Ep

102/102 [==============================] - 0s 3ms/step - loss: 2.4170 - accuracy: 0.6242
Epoch 44/50
102/102 [==============================] - 0s 3ms/step - loss: 2.4039 - accuracy: 0.6226
Epoch 45/50
102/102 [==============================] - 0s 3ms/step - loss: 2.3912 - accuracy: 0.6257
Epoch 46/50
102/102 [==============================] - 0s 3ms/step - loss: 2.3778 - accuracy: 0.6251
Epoch 47/50
102/102 [==============================] - 0s 3ms/step - loss: 2.3655 - accuracy: 0.6280
Epoch 48/50
102/102 [==============================] - 0s 3ms/step - loss: 2.3538 - accuracy: 0.6283
Epoch 49/50
102/102 [==============================] - 0s 3ms/step - loss: 2.3428 - accuracy: 0.6308
Epoch 50/50
102/102 [==============================] - 0s 3ms/step - loss: 2.3316 - accuracy: 0.6306

Current Epoch: 50
Epoch 1/50
102/102 [==============================] - 0s 3ms/step - loss: 5.5189 - accuracy: 0.3234
Epoch 2/50
102/102 [==============================] - 0s 3ms/step - loss: 5.1294 - ac

102/102 [==============================] - 0s 3ms/step - loss: 2.8072 - accuracy: 0.5863
Epoch 25/50
102/102 [==============================] - 0s 3ms/step - loss: 2.7769 - accuracy: 0.5863
Epoch 26/50
102/102 [==============================] - 0s 3ms/step - loss: 2.7478 - accuracy: 0.5948
Epoch 27/50
102/102 [==============================] - 0s 3ms/step - loss: 2.7213 - accuracy: 0.5938
Epoch 28/50
102/102 [==============================] - 0s 3ms/step - loss: 2.6950 - accuracy: 0.5968
Epoch 29/50
102/102 [==============================] - 0s 3ms/step - loss: 2.6710 - accuracy: 0.6003
Epoch 30/50
102/102 [==============================] - 0s 3ms/step - loss: 2.6476 - accuracy: 0.6000
Epoch 31/50
102/102 [==============================] - 0s 3ms/step - loss: 2.6255 - accuracy: 0.6012
Epoch 32/50
102/102 [==============================] - 0s 3ms/step - loss: 2.6044 - accuracy: 0.6060
Epoch 33/50
102/102 [==============================] - 0s 3ms/step - loss: 2.5833 - accuracy: 0.6052
Ep

...

102/102 [==============================] - 0s 3ms/step - loss: 1.8414 - accuracy: 0.7254
Epoch 30/50
102/102 [==============================] - 0s 3ms/step - loss: 1.8254 - accuracy: 0.7289
Epoch 31/50
102/102 [==============================] - 0s 3ms/step - loss: 1.8097 - accuracy: 0.7314
Epoch 32/50
102/102 [==============================] - 0s 3ms/step - loss: 1.7940 - accuracy: 0.7305
Epoch 33/50
102/102 [==============================] - 0s 3ms/step - loss: 1.7791 - accuracy: 0.7317
Epoch 34/50
102/102 [==============================] - 0s 3ms/step - loss: 1.7640 - accuracy: 0.7351
Epoch 35/50
102/102 [==============================] - 0s 3ms/step - loss: 1.7488 - accuracy: 0.7315
Epoch 36/50
102/102 [==============================] - 0s 3ms/step - loss: 1.7357 - accuracy: 0.7323
Epoch 37/50
102/102 [==============================] - 0s 3ms/step - loss: 1.7210 - accuracy: 0.7355
Epoch 38/50
102/102 [==============================] - 0s 3ms/step - loss: 1.7080 - accuracy: 0.7366
Ep

102/102 [==============================] - 0s 3ms/step - loss: 2.2986 - accuracy: 0.6966
Epoch 11/50
102/102 [==============================] - 0s 3ms/step - loss: 2.2461 - accuracy: 0.7037
Epoch 12/50
102/102 [==============================] - 0s 3ms/step - loss: 2.2022 - accuracy: 0.7086
Epoch 13/50
102/102 [==============================] - 0s 3ms/step - loss: 2.1639 - accuracy: 0.7128
Epoch 14/50
102/102 [==============================] - 0s 3ms/step - loss: 2.1281 - accuracy: 0.7154
Epoch 15/50
102/102 [==============================] - 0s 3ms/step - loss: 2.0939 - accuracy: 0.7166
Epoch 16/50
102/102 [==============================] - 0s 3ms/step - loss: 2.0647 - accuracy: 0.7218
Epoch 17/50
102/102 [==============================] - 0s 3ms/step - loss: 2.0364 - accuracy: 0.7206
Epoch 18/50
102/102 [==============================] - 0s 3ms/step - loss: 2.0108 - accuracy: 0.7228
Epoch 19/50
102/102 [==============================] - 0s 3ms/step - loss: 1.9849 - accuracy: 0.7249
Ep

102/102 [==============================] - 0s 3ms/step - loss: 1.6137 - accuracy: 0.7477
Epoch 42/50
102/102 [==============================] - 0s 3ms/step - loss: 1.6027 - accuracy: 0.7463
Epoch 43/50
102/102 [==============================] - 0s 3ms/step - loss: 1.5919 - accuracy: 0.7457
Epoch 44/50
102/102 [==============================] - 0s 2ms/step - loss: 1.5794 - accuracy: 0.7475
Epoch 45/50
102/102 [==============================] - 0s 3ms/step - loss: 1.5693 - accuracy: 0.7492
Epoch 46/50
102/102 [==============================] - 0s 2ms/step - loss: 1.5577 - accuracy: 0.7494
Epoch 47/50
102/102 [==============================] - 0s 3ms/step - loss: 1.5472 - accuracy: 0.7488
Epoch 48/50
102/102 [==============================] - 0s 3ms/step - loss: 1.5377 - accuracy: 0.7505
Epoch 49/50
102/102 [==============================] - 0s 3ms/step - loss: 1.5284 - accuracy: 0.7488
Epoch 50/50
102/102 [==============================] - 0s 3ms/step - loss: 1.5181 - accuracy: 0.7515

C

102/102 [==============================] - 0s 3ms/step - loss: 1.9277 - accuracy: 0.7220
Epoch 23/50
102/102 [==============================] - 0s 3ms/step - loss: 1.9065 - accuracy: 0.7278
Epoch 24/50
102/102 [==============================] - 0s 3ms/step - loss: 1.8876 - accuracy: 0.7268
Epoch 25/50
102/102 [==============================] - 0s 3ms/step - loss: 1.8676 - accuracy: 0.7302
Epoch 26/50
102/102 [==============================] - 0s 3ms/step - loss: 1.8489 - accuracy: 0.7314
Epoch 27/50
102/102 [==============================] - 0s 3ms/step - loss: 1.8315 - accuracy: 0.7314
Epoch 28/50
102/102 [==============================] - 0s 3ms/step - loss: 1.8133 - accuracy: 0.7317
Epoch 29/50
102/102 [==============================] - 0s 3ms/step - loss: 1.7959 - accuracy: 0.7331
Epoch 30/50
102/102 [==============================] - 0s 3ms/step - loss: 1.7797 - accuracy: 0.7360
Epoch 31/50
102/102 [==============================] - 0s 3ms/step - loss: 1.7645 - accuracy: 0.7365
Ep

102/102 [==============================] - 0s 3ms/step - loss: 4.3627 - accuracy: 0.2606
Epoch 3/50
102/102 [==============================] - 0s 3ms/step - loss: 3.5509 - accuracy: 0.4986
Epoch 4/50
102/102 [==============================] - 0s 3ms/step - loss: 3.0754 - accuracy: 0.5988
Epoch 5/50
102/102 [==============================] - 0s 3ms/step - loss: 2.7961 - accuracy: 0.6412
Epoch 6/50
102/102 [==============================] - 0s 3ms/step - loss: 2.6206 - accuracy: 0.6658
Epoch 7/50
102/102 [==============================] - 0s 3ms/step - loss: 2.4969 - accuracy: 0.6838
Epoch 8/50
102/102 [==============================] - 0s 3ms/step - loss: 2.4086 - accuracy: 0.6922
Epoch 9/50
102/102 [==============================] - 0s 3ms/step - loss: 2.3380 - accuracy: 0.6940
Epoch 10/50
102/102 [==============================] - 0s 3ms/step - loss: 2.2806 - accuracy: 0.7011
Epoch 11/50
102/102 [==============================] - 0s 3ms/step - loss: 2.2311 - accuracy: 0.7057
Epoch 12/

In [7]:
result

{0: [0.010461538098752499, 0.015076923184096813],
 10: [0.517230749130249, 0.520307719707489],
 20: [0.5973846316337585, 0.5979999899864197],
 30: [0.6246153712272644, 0.6246153712272644],
 40: [0.6306154131889343, 0.6307692527770996],
 50: [0.6359999775886536, 0.6359999775886536],
 60: [0.6295384764671326, 0.6295384764671326],
 70: [0.7203077077865601, 0.7203077077865601],
 80: [0.7069230675697327, 0.7081538438796997],
 90: [0.7101538181304932, 0.7101538181304932],
 100: [0.7135384678840637, 0.7139999866485596],
 110: [0.7056922912597656, 0.7056922912597656],
 120: [0.6984615325927734, 0.7001538276672363],
 130: [0.7492307424545288, 0.7492307424545288],
 140: [0.7456923127174377, 0.7456923127174377],
 150: [0.7510769367218018, 0.7526153922080994],
 160: [0.7495384812355042, 0.7495384812355042],
 170: [0.7515384554862976, 0.7515384554862976],
 178: [0.7532307505607605, 0.7532307505607605],
 180: [0.7526153922080994, 0.7526153922080994],
 190: [0.7509230971336365, 0.7509230971336365],
 